In [1]:
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')

df_1_3 = pd.read_csv('results/main_table.tsv',  sep='\t')
df_4_bgpd_gbm = pd.read_csv('results/transfer_bgpd_gbm.tsv',  sep='\t')
df_4_gbm_bgpd = pd.read_csv('results/transfer_gbm_bgpd.tsv',  sep='\t')

df_1_3.replace([np.inf, -np.inf], np.nan, inplace=True)

In [2]:
df_1_3.head()

,PatientID,Preprocessing step,Dataset,"NN-Architecture (nn-Unet, UNETR)","Label (WT, ET, TC, GTV)",Dice,Hausdorff,Surface Dice 1mm,Surface Dice 3mm,Surface Dice 5mm,Epoch,Volume_estimated,Volume_intersection,Volume_true
0,TCGA-02-0011,1_reg,gbm,nn_unet,WT,0.290,39.000,0.264,NaN,NaN,300,25.748,22.114,126.783
1,TCGA-02-0011,1_reg,gbm,nn_unet,ET,0.138,39.000,0.218,NaN,NaN,300,2.983,22.114,33.488
2,TCGA-02-0011,1_reg,gbm,nn_unet,TC,0.030,34.300,0.133,NaN,NaN,300,9.525,22.114,81.619
3,TCGA-02-0033,1_reg,gbm,nn_unet,WT,0.242,39.000,0.261,NaN,NaN,300,19.781,14.996,104.407
4,TCGA-02-0033,1_reg,gbm,nn_unet,ET,0.265,26.094,0.330,NaN,NaN,300,6.324,14.996,31.703


In [3]:
## Table 4: 

reindex_list = ['1_reg', '2a_interp', '3a_atlas', '4a_resamp', '4b_n4', '4d_susan','6_hist','5_ss_shared']
rename_list = ['1. Inter modality', '2. Resampling', '3. Atlas', '4. Resampling to spacing', '4.a Bias', '4.b Denoising', '4.c Histogram', '4.d Skull']


In [4]:
df_1_3.Dataset.unique()

array(['gbm', 'bgpd', 'lgg'], dtype=object)

По каждой метрике проверяем на gbm, bgpd, lgg. Делаем для nnU-net и unetr.

In [5]:
#Table 4 DICE WT
models = df_1_3['NN-Architecture (nn-Unet, UNETR)'].unique()
metric = 'Dice'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300
# to save space in the paper and not write 0.44 and write 44 instead (only for Dice and Surface Dice)
multiplier = 100


def get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier):
    
    # mean
    temp_series = (np.round(df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model ].groupby([ 'Preprocessing step'])[metric].mean(),3)*multiplier).round(0)
    temp_series = temp_series.reindex(reindex_list)
    temp_series.index = rename_list
    temp_series_mean = temp_series

    #std
    temp_series = (np.round(df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model].groupby([ 'Preprocessing step'])[metric].std(),3)*multiplier).round(0)

    temp_series = temp_series.reindex(reindex_list)
    temp_series.index = rename_list
    temp_series_std = temp_series

    temp_mean_std = pd.concat([temp_series_mean, temp_series_std], axis=1)
    temp_mean_std.columns = ['Mean', 'Std']
    temp_mean_std['Mean (STD)'] = temp_mean_std.apply(lambda x: str(x['Mean']) + ' (' + str(x['Std']) + ')', axis=1)
    return temp_mean_std['Mean (STD)'] 

print('DICE scores')
for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        print(data, model, metric)
        print(get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier))
        print( )

DICE scores
=======================NEW MODEL=======================
gbm nn_unet Dice
1. Inter modality           44.0 (28.0)
2. Resampling               85.0 (11.0)
3. Atlas                    85.0 (11.0)
4. Resampling to spacing    85.0 (12.0)
4.a Bias                    82.0 (13.0)
4.b Denoising               84.0 (12.0)
4.c Histogram               83.0 (16.0)
4.d Skull                   87.0 (11.0)
Name: Mean (STD), dtype: object

bgpd nn_unet Dice
1. Inter modality           36.0 (29.0)
2. Resampling               73.0 (19.0)
3. Atlas                    75.0 (16.0)
4. Resampling to spacing    74.0 (18.0)
4.a Bias                    75.0 (17.0)
4.b Denoising               74.0 (17.0)
4.c Histogram               75.0 (16.0)
4.d Skull                   76.0 (14.0)
Name: Mean (STD), dtype: object

lgg nn_unet Dice
1. Inter modality           67.0 (27.0)
2. Resampling               72.0 (24.0)
3. Atlas                    71.0 (25.0)
4. Resampling to spacing    70.0 (25.0)
4.a Bias      

In [6]:
# Appending Table  Hausdorff WT
metric = 'Hausdorff'
data = 'gbm'
label = 'WT'
models = df_1_3['NN-Architecture (nn-Unet, UNETR)'].unique()

#get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier=1)

print('Hausdorff scores')
for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        print(data, model, metric)
        print(get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier=1))
        print( )

Hausdorff scores
=======================NEW MODEL=======================
gbm nn_unet Hausdorff
1. Inter modality           31.0 (21.0)
2. Resampling               15.0 (21.0)
3. Atlas                    10.0 (11.0)
4. Resampling to spacing    13.0 (20.0)
4.a Bias                    14.0 (22.0)
4.b Denoising               11.0 (16.0)
4.c Histogram               13.0 (18.0)
4.d Skull                   10.0 (14.0)
Name: Mean (STD), dtype: object

bgpd nn_unet Hausdorff
1. Inter modality           46.0 (39.0)
2. Resampling               26.0 (33.0)
3. Atlas                    22.0 (27.0)
4. Resampling to spacing    23.0 (34.0)
4.a Bias                    22.0 (34.0)
4.b Denoising               26.0 (39.0)
4.c Histogram               22.0 (34.0)
4.d Skull                   15.0 (17.0)
Name: Mean (STD), dtype: object

lgg nn_unet Hausdorff
1. Inter modality           35.0 (36.0)
2. Resampling               29.0 (32.0)
3. Atlas                    27.0 (35.0)
4. Resampling to spacing    38.0 (

In [7]:
# Appending Table  Surface Dice WT
metric = 'Surface Dice 1mm'
data = 'gbm'
label = 'WT'
model = 'nn_unet'

print('Surface Dice WT')
for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        print(data, model, metric)
        print(get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier=100))
        print( )

Surface Dice WT
=======================NEW MODEL=======================
gbm nn_unet Surface Dice 1mm
1. Inter modality           27.0 (15.0)
2. Resampling               65.0 (20.0)
3. Atlas                    63.0 (20.0)
4. Resampling to spacing    65.0 (20.0)
4.a Bias                    60.0 (22.0)
4.b Denoising               64.0 (21.0)
4.c Histogram               63.0 (22.0)
4.d Skull                   69.0 (20.0)
Name: Mean (STD), dtype: object

bgpd nn_unet Surface Dice 1mm
1. Inter modality           20.0 (14.0)
2. Resampling               41.0 (18.0)
3. Atlas                    44.0 (17.0)
4. Resampling to spacing    43.0 (18.0)
4.a Bias                    44.0 (19.0)
4.b Denoising               42.0 (18.0)
4.c Histogram               43.0 (18.0)
4.d Skull                   45.0 (19.0)
Name: Mean (STD), dtype: object

lgg nn_unet Surface Dice 1mm
1. Inter modality           41.0 (24.0)
2. Resampling               47.0 (24.0)
3. Atlas                    46.0 (26.0)
4. Resampling 

In [13]:
## How to check where to put rows on the statistical significance:

def dropna_intersection(data_1: pd.Series, data_2: pd.Series):
    data_1 = data_1.reset_index(drop=True)
    data_2 = data_2.reset_index(drop=True)

    data_1 = data_1.dropna()
    data_2 = data_2.dropna()

    data_1 = data_1[data_1.index.isin(data_2.index)]
    data_2 = data_2[data_2.index.isin(data_1.index)]
    
    return data_1, data_2

In [14]:
import scipy
from statsmodels.stats.weightstats import ttost_ind, ttost_paired

metric = 'Hausdorff'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300


preproc_1 = "4a_resamp"
for key in reindex_list:
    preproc_2 = key
    
    if key != "4a_resamp":

        data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
        data_2 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_2][metric]

        
        ttost = ttost_paired(np.array(data_1), np.array(data_2), -data_2.std()/4, data_2.std()/4)[0]/7
        ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7

        # print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
        if ttost > 0.05:
            print(preproc_2, (np.round(data_1.mean(),3)*1).round(0).astype(int), '(',
              (np.round(data_1.std(),3)*1).round(0).astype(int), ')',
              ' vs ',\
              (np.round(data_2.mean(),3)*1).round(0).astype(int), '(' ,
              (np.round(data_2.std(),3)*1).round(0).astype(int),')')
            print( "Difference is significant, p-value ", np.round(ttest_,3))

1_reg 13 ( 20 )  vs  31 ( 21 )
Difference is significant, p-value  0.0
3a_atlas 13 ( 20 )  vs  10 ( 11 )
Difference is significant, p-value  6.359
5_ss_shared 13 ( 20 )  vs  10 ( 14 )
Difference is significant, p-value  0.104


In [34]:
10/3.8

2.6315789473684212

In [35]:
metric = 'Dice'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300

for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        preproc_1 = "4a_resamp"
        for key in reindex_list:
            preproc_2 = key

            if key != "4a_resamp":

                data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
                data_2 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_2][metric]
                data_1, data_2 = dropna_intersection(data_1, data_2)

                ttost = ttost_paired(np.array(data_1), np.array(data_2), -data_2.std()/10, data_2.std()/10)[0]/7
                ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7
                                
                #assert isna
                assert data_1.isnull().sum() == 0
                assert data_2.isnull().sum() == 0

                # print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
                if ttost > 0.05:
                    print(data, model, metric)
                    print(preproc_2, (np.round(data_1.mean(),3)*100).round(0).astype(int), '(',
                      (np.round(data_1.std(),3)*100).round(0).astype(int), ')',
                      ' vs ',\
                      (np.round(data_2.mean(),3)*100).round(0).astype(int), '(' ,
                      (np.round(data_2.std(),3)*100).round(0).astype(int),')')
                    print( "Wilcoxon ", np.round(ttest_,3), 'TTOST', np.round(ttost,3))
                    print( )

=======================NEW MODEL=======================
gbm nn_unet Dice
1_reg 85 ( 12 )  vs  44 ( 28 )
Wilcoxon  0.0 TTOST 0.143

gbm nn_unet Dice
4b_n4 85 ( 12 )  vs  82 ( 13 )
Wilcoxon  0.0 TTOST 0.134

gbm nn_unet Dice
6_hist 85 ( 12 )  vs  83 ( 16 )
Wilcoxon  0.608 TTOST 0.074

gbm nn_unet Dice
5_ss_shared 85 ( 12 )  vs  87 ( 11 )
Wilcoxon  0.019 TTOST 0.113

bgpd nn_unet Dice
1_reg 74 ( 18 )  vs  36 ( 29 )
Wilcoxon  0.0 TTOST 0.143

bgpd nn_unet Dice
3a_atlas 74 ( 18 )  vs  75 ( 16 )
Wilcoxon  0.914 TTOST 0.066



bgpd nn_unet Dice
5_ss_shared 74 ( 18 )  vs  76 ( 14 )
Wilcoxon  0.013 TTOST 0.126

lgg nn_unet Dice
1_reg 70 ( 25 )  vs  67 ( 27 )
Wilcoxon  0.0 TTOST 0.108

lgg nn_unet Dice
4b_n4 70 ( 25 )  vs  67 ( 25 )
Wilcoxon  0.219 TTOST 0.102

lgg nn_unet Dice
6_hist 70 ( 25 )  vs  68 ( 26 )
Wilcoxon  1.498 TTOST 0.072

lgg nn_unet Dice
5_ss_shared 70 ( 25 )  vs  77 ( 21 )
Wilcoxon  0.0 TTOST 0.142

=======================NEW MODEL=======================
gbm UNETR Dice
1_reg 82 ( 14 )  vs  39 ( 26 )
Wilcoxon  0.0 TTOST 0.143

gbm UNETR Dice
4b_n4 82 ( 14 )  vs  80 ( 13 )
Wilcoxon  0.0 TTOST 0.121

gbm UNETR Dice
6_hist 82 ( 14 )  vs  81 ( 16 )
Wilcoxon  2.673 TTOST 0.062

gbm UNETR Dice
5_ss_shared 82 ( 14 )  vs  85 ( 10 )
Wilcoxon  0.158 TTOST 0.13

bgpd UNETR Dice
1_reg 67 ( 21 )  vs  34 ( 30 )
Wilcoxon  0.0 TTOST 0.143

bgpd UNETR Dice
4b_n4 67 ( 21 )  vs  72 ( 19 )
Wilcoxon  0.0 TTOST 0.143

bgpd UNETR Dice
4d_susan 67 ( 21 )  vs  69 ( 21 )
Wilcoxon  0.0 TTOST 0.066

bgpd UNETR Dice
5_ss_s

In [22]:
metric = 'Hausdorff'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300

for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        preproc_1 = "4a_resamp"

        for key in reindex_list:
            preproc_2 = key

            if key != "4a_resamp":

                data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
                data_2 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_2][metric]
 
                data_1, data_2 = dropna_intersection(data_1, data_2)
                
                #assert isna
                assert data_1.isnull().sum() == 0
                assert data_2.isnull().sum() == 0
                
                ttost = ttost_paired(np.array(data_1), np.array(data_2), -data_2.std()/4, data_2.std()/4)[0]/7
                ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7

                # print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
                if  (ttost > 0.05):
                    print(data, model, metric)
                    print(preproc_2, (np.round(data_1.mean(),3)*1).round(0).astype(int), '(',
                      (np.round(data_1.std(),3)*1).round(0).astype(int), ')',
                      ' vs ',\
                      (np.round(data_2.mean(),3)*1).round(0).astype(int), '(' ,
                      (np.round(data_2.std(),3)*1).round(0).astype(int),')')
                    print( "Difference is significant, p-value ", np.round(ttest_,3), np.round(ttost,3))
                    print( )

=======================NEW MODEL=======================
gbm nn_unet Hausdorff
1_reg 13 ( 20 )  vs  31 ( 21 )
Difference is significant, p-value  0.0 0.143

gbm nn_unet Hausdorff
3a_atlas 13 ( 20 )  vs  10 ( 11 )
Difference is significant, p-value  6.359 0.08

gbm nn_unet Hausdorff
5_ss_shared 13 ( 20 )  vs  10 ( 14 )
Difference is significant, p-value  0.104 0.055

bgpd nn_unet Hausdorff
1_reg 24 ( 35 )  vs  46 ( 39 )
Difference is significant, p-value  0.0 0.143

bgpd nn_unet Hausdorff
5_ss_shared 23 ( 34 )  vs  15 ( 17 )
Difference is significant, p-value  0.025 0.132

lgg nn_unet Hausdorff
2a_interp 38 ( 39 )  vs  29 ( 32 )
Difference is significant, p-value  0.037 0.084

lgg nn_unet Hausdorff
3a_atlas 38 ( 39 )  vs  27 ( 35 )
Difference is significant, p-value  0.213 0.109

lgg nn_unet Hausdorff
5_ss_shared 38 ( 39 )  vs  23 ( 25 )
Difference is significant, p-value  0.001 0.142

=======================NEW MODEL=======================
gbm UNETR Hausdorff
1_reg 16 ( 23 )  vs  49 ( 2

gbm UNETR Hausdorff
4d_susan 16 ( 23 )  vs  26 ( 31 )
Difference is significant, p-value  0.068 0.118

gbm UNETR Hausdorff
6_hist 16 ( 23 )  vs  26 ( 31 )
Difference is significant, p-value  1.229 0.115

gbm UNETR Hausdorff
5_ss_shared 16 ( 23 )  vs  12 ( 16 )
Difference is significant, p-value  0.001 0.075

bgpd UNETR Hausdorff
1_reg 34 ( 35 )  vs  42 ( 31 )
Difference is significant, p-value  0.001 0.083

bgpd UNETR Hausdorff
2a_interp 34 ( 36 )  vs  43 ( 33 )
Difference is significant, p-value  0.002 0.073

bgpd UNETR Hausdorff
4b_n4 34 ( 36 )  vs  27 ( 33 )
Difference is significant, p-value  0.001 0.052

bgpd UNETR Hausdorff
5_ss_shared 34 ( 36 )  vs  20 ( 21 )
Difference is significant, p-value  0.0 0.143

lgg UNETR Hausdorff
1_reg 44 ( 36 )  vs  35 ( 31 )
Difference is significant, p-value  0.456 0.089

lgg UNETR Hausdorff
2a_interp 44 ( 36 )  vs  36 ( 35 )
Difference is significant, p-value  2.502 0.058

lgg UNETR Hausdorff
5_ss_shared 44 ( 36 )  vs  25 ( 28 )
Difference is sig

In [18]:
metric = 'Surface Dice 1mm'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300

for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        preproc_1 = "4a_resamp"
        for key in reindex_list:
            preproc_2 = key

            if key != "4a_resamp":

                data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
                data_2 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_2][metric]
                data_1, data_2 = dropna_intersection(data_1, data_2)
                
                #assert isna
                assert data_1.isnull().sum() == 0
                assert data_2.isnull().sum() == 0

                ttost = ttost_ind(data_1, data_2, -data_2.std()/5, data_2.std()/5)[0]/7
                ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7

                # print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
                if ttost > 0.05:
                    print(data, model, metric)
                    print(preproc_2, (np.round(data_1.mean(),3)*100).round(0).astype(int), '(',
                      (np.round(data_1.std(),3)*100).round(0).astype(int), ')',
                      ' vs ',\
                      (np.round(data_2.mean(),3)*100).round(0).astype(int), '(' ,
                      (np.round(data_2.std(),3)*100).round(0).astype(int),')')
                    print( "Difference is significant, p-value ", np.round(ttest_,3))
                    print( )

=======================NEW MODEL=======================
gbm nn_unet Surface Dice 1mm
1_reg 65 ( 20 )  vs  27 ( 15 )
Difference is significant, p-value  0.0

gbm nn_unet Surface Dice 1mm
4b_n4 65 ( 20 )  vs  60 ( 22 )
Difference is significant, p-value  0.0

gbm nn_unet Surface Dice 1mm
5_ss_shared 65 ( 20 )  vs  69 ( 20 )
Difference is significant, p-value  0.037

bgpd nn_unet Surface Dice 1mm
1_reg 43 ( 18 )  vs  20 ( 14 )
Difference is significant, p-value  0.0

lgg nn_unet Surface Dice 1mm
1_reg 46 ( 25 )  vs  41 ( 24 )
Difference is significant, p-value  0.0

lgg nn_unet Surface Dice 1mm
4b_n4 46 ( 25 )  vs  42 ( 25 )
Difference is significant, p-value  0.017

lgg nn_unet Surface Dice 1mm
6_hist 46 ( 25 )  vs  42 ( 24 )
Difference is significant, p-value  0.157

lgg nn_unet Surface Dice 1mm
5_ss_shared 46 ( 25 )  vs  52 ( 23 )
Difference is significant, p-value  0.004

=======================NEW MODEL=======================
gbm UNETR Surface Dice 1mm
1_reg 59 ( 21 )  vs  22 ( 12 )
